In [5]:
import pandas as pd
import numpy as np
import cufflinks as cf

from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

def parse_ts(ts):
    return pd.to_datetime(ts, unit="ms", utc=False)

headers = ['ts', 'X', 'Y', 'Z']
df = pd.read_csv('data/wash_idle.csv', names=headers, \
                 parse_dates=True,date_parser=parse_ts, index_col='ts')
df = df.drop(columns=["Z", "Y"])
iplot(df.iplot(asFigure=True, kind='scatter'))

In [6]:
len(df)

6901

In [7]:
import pandas as pd
import numpy as np
import cufflinks as cf

from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

LIMIT=3.0

l1 = lambda x: x if x < LIMIT else LIMIT

def parse_ts(ts):
    return pd.to_datetime(ts, unit="ms", utc=False)

def slice_stddev(df, batch_size, cols, aggregator=np.std):
    sddf = pd.DataFrame(columns=cols)
    sddf.set_index('ts', inplace=True)
    batches = int(len(df)/batch_size)
    for i in range(batches+1):
        start = i*batch_size
        end = (i+1)*batch_size
        slice = df[start:end] if end < len(df) else df[start:]
        if len(slice) < 1:
            continue
        ts = parse_ts(slice['ts'].values[0])
        sddf.loc[ts] = [
            l1(aggregator(slice['X'].values)),
            l1(aggregator(slice['Y'].values)),
            l1(aggregator(slice['Z'].values))
        ]
    return sddf

In [10]:
headers = ['ts', 'X', 'Y', 'Z']
df = pd.read_csv('data/wash_run1.csv', names=headers)

sdf = slice_stddev(df, 200, headers, aggregator=np.std)
sdf = sdf.drop(columns=["Z", "Y"])
iplot(sdf.iplot(asFigure=True, kind='scatter'))